# NLI eval

In [ ]:
# use horovod pytorch
from __future__ import absolute_import, division, print_function

import argparse
import csv
import json
import logging
import os
import random
import sys

import numpy as np
import torch
import torch.nn.functional as F

# mult processing
import torch.multiprocessing as mp

# for horovod distributed training
import torch.utils.data.distributed

from transformers import (WEIGHTS_NAME, AdamW, BertConfig,
                          BertForTokenClassification, BertTokenizer,
                          get_linear_schedule_with_warmup, 
                          BertPreTrainedModel, BertModel) # cls classifier 를 위해 로드

# from transformers.models.bert.modeling_bert import Sentence_classifier
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from torch import nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
# from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from seqeval.metrics import classification_report
from seqeval.metrics import sequence_labeling # 따로 결과를 뽑아내고 싶음

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)
import codecs

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
import glob
import time

import easydict

import six
import tqdm
from tqdm import tqdm, trange
import collections

from seqeval.metrics import classification_report

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


import re

import shutil

In [ ]:
class TrainingInstance(object):
    """A single training instance (sentence pair)."""
    def __init__(self, input_ids, input_mask, segment_ids, is_entailed):
        self.input_ids=input_ids, 
        self.input_mask=input_mask, 
        self.segment_ids=segment_ids, 
        self.is_entailed=is_entailed

## CLS classifier

In [ ]:
class CLS_classifier(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, 8)
        self.init_weights()
        
    def forward(self, input_ids, label_ids, 
                attention_mask=None, token_type_ids=None,  
                position_ids=None, head_mask=None, inputs_embeds=None,
                output_attentions=None, output_hidden_states=None,
                return_dict=None):
        
        outputs = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            position_ids = position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        pooled_output = outputs[1]
        
        seq_relationship_scores = self.classifier(pooled_output)
        batsize = seq_relationship_scores.size()[0]
        
        
        # labels
        labels = torch.zeros(batsize, dtype=torch.long, device='cuda')
        for l in range(len(labels)):
            labels[l]=label_ids[l][0]
        
        loss_fct = CrossEntropyLoss()
        
        # label 8
        loss = loss_fct(seq_relationship_scores.view(-1, 8), labels.view(-1))
        
        probs = seq_relationship_scores.detach().cpu()
        probs = nn.functional.log_softmax(probs, dim=-1) 
        
        return loss, probs

## Evaluation

In [ ]:
def evaluate(args):
    ############################################################################
    ##              Multi-GPUs, Distributed settings
    ############################################################################ 
    if args.local_rank == -1 or args.no_cuda:
        #device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        #n_gpu = torch.cuda.device_count()
        device = "cuda:0"
        n_gpu = 1
        
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))
    
    
    ###############################################################
    #              Functions for Dataload (load cache data)
    ###############################################################
    def get_file_arrays(data_path):
        data_path = data_path.split(",")
        print("data_path: ", data_path)
        
        cache_files_all = []
        for i in range(len(data_path)):
            path = data_path[i].strip()
            files = glob.glob(path)
            cache_files_all = cache_files_all + files
        cache_files_all.sort()
        
        print("Got " + str(len(cache_files_all)) + " cache_files")
        
        return cache_files_all
    
    
    def read_files(cache_files_pieces):
        eval_features = []
        for c in range(len(cache_files_pieces)):
            with open(cache_files_pieces[c], 'rb') as input:
                eval_features = eval_features + pickle.load(input)      
        
        # 학습 샘플의 개수
        num_eval_examples = len(eval_features)
        
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_labels = torch.tensor([f.labels for f in eval_features], dtype=torch.long)
        
        # 반드시 squeeeze 하여 돌릴 것 (쓸데없는 차원떄문에 모델 forward 가 안되는 문제)
        all_input_ids = torch.squeeze(all_input_ids)
        all_input_mask = torch.squeeze(all_input_mask)
        all_segment_ids = torch.squeeze(all_segment_ids)
        all_labels = torch.squeeze(all_labels)

        eval_data = TensorDataset(
            all_input_ids, all_input_mask, all_segment_ids, all_labels
        )
        
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
    
        return eval_dataloader, num_eval_examples
    
    ############################################################################
    ##                    labels
    ############################################################################
    label_list = ["0", "1", "2", "3", "4", "5", "6", "7"] # 8 classes
    num_labels = len(label_list) # + 1 NER 에서는 [PAD] 때문에 +1 했었음
    

    ############################################################################
    ##                      Prepare model
    ############################################################################
    print("loading weights from checkpoint (", args.checkpoint, ")")
    config = BertConfig.from_pretrained(args.checkpoint, num_labels=num_labels)
    model = CLS_classifier.from_pretrained(args.checkpoint,
              from_tf = False,
              config = config)
    print("loaded weights from checkpoint (", args.checkpoint, ")")
    
    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    print("device: ", device)
    
    model.to(device)
    
    ############################################################################
    ##              Start evalution
    ############################################################################
    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0
    
    model.eval()
    
    
    ###############################################################
    #            Dataload (load cache data) - Train data
    ###############################################################
    cache_files = get_file_arrays(args.data_dir)
    print("cache_files: ", cache_files)
    
    true_pred = 0 # true prediction
    true_gold = 0 # true label
    
    true_positives = 0
    num_samples = 0
    
    for c in range(len(cache_files)):
        print("targetfile: ", cache_files[c])
        eval_dataloader, num_eval_examples = read_files([cache_files[c]])
        
        filename = cache_files[c].split("/")[-1]
        
        outtext = "label\tprediction\n"
        
        for step, batch in enumerate(tqdm(eval_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch

            loss, probs = model(input_ids=input_ids, label_ids=label_ids, 
                attention_mask=input_mask, token_type_ids=segment_ids,
                position_ids=None, head_mask=None, inputs_embeds=None,
                output_attentions=None, output_hidden_states=None,
                return_dict=None)
            
            for b in range(len(probs)):
                label = int(label_ids[b][0].item())
                
                pred = int(torch.argmax(probs[b], dim=-1).item())
             
                outtext = outtext + str(label)+"\t"
                outtext = outtext + str(pred)+"\n"
                num_samples = num_samples + 1
                
                
                if pred==label:
                    true_pred = true_pred + 1
                    
                
        # output predictions
        if not os.path.exists(args.output_dir+"/preds/"):
            os.makedirs(args.output_dir+"/preds/")
        
        file = open(args.output_dir+"/preds/"+filename[:-len(".cache")]+".txt", "w")
        file.write(outtext)
        file.close()
        
    # output score
    file = open(args.output_dir+"/dptment_accuracy.txt", "w")
    
    scoretext = "accuracy\n"
    scoretext = str(true_pred/num_samples)
    
    file.write(scoretext)
    file.close()
    
    print('str(true_pred/num_samples): ', str(true_pred/num_samples))
    

In [ ]:
def main(args):
    parser = argparse.ArgumentParser()
    
    print("args.output_dir: ", args.output_dir)
    
    # check directories
    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
        
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    
    if args.do_train:
        train(args)
    
    if args.do_eval:
        evaluate(args)

# Evaluate Test set

In [ ]:
data_dirs = [
    './cache/mbert_cased/test/*.cache',
    './cache/bertbase_cased/test/*.cache',
    './cache/biobert/test/*.cache',    
    './cache/kobert/test/*.cache',
]

# pretrained models
checkpoints = [
    "./finetuned/ver8.1.4_1142642_epoch2/3199",
    "./finetuned/ver9.1.4_521121_epoch2/3199",
    './finetuned/ver11.1.4_521079_epoch2/3199',
    './finetuned/ver12.1.4_407013_epoch2/3199',
]

assert len(data_dirs)==len(checkpoints)

for d in range(len(data_dirs)):
    args = easydict.EasyDict({
        'data_dir':data_dirs[d],
        'eval_batch_size':4,             # train_batch_size/gradient_accumulation_steps
        'gradient_accumulation_steps':1,  # become update step
        'save_globalstep':1,             
        'checkpoint':checkpoints[d],
        'output_dir':checkpoints[d],
        'num_files_on_memory':2,

        'num_train_epochs':2,
        'warmup_proportion':0.5,
        'do_train':False,
        'do_eval':True,
        'do_lower_case':True,
        'fp16_allreduce':False,
        'seed':42,
        'no_cuda':False,
        'use_adasum':False,
        'learning_rate':0.00005,
        'weight_decay':0.01,
        'adam_epsilon':1e-8,
        'gradient_predivide_factor':1.0,
        'fp16':False,
        'max_grad_norm':1.0,
        'num_files_on_memory':2,
        'local_rank':-1,
        'fp16_opt_level':'O1',
        'loss_scale':0,
        'server_ip':'',
        'server_port':'',    
    })

    main(args)